# Multi-Agent Supervisor with Azure OpenAI

This notebook demonstrates how to create a multi-agent system using LangGraph with a supervisor agent that delegates tasks to specialized agents. The system uses Azure OpenAI for all language model operations.

## Overview

The system consists of:
1. **Supervisor Agent**: Manages task delegation and coordination
2. **Research Agent**: Handles research-related tasks
3. **Math Agent**: Handles mathematical calculations
4. **Writing Agent**: Handles content creation and writing tasks

## Prerequisites

Before running this notebook, ensure you have:
1. An Azure OpenAI resource with deployed models
2. The required Python packages installed
3. Your Azure OpenAI credentials configured


## 1. Installation and Setup


In [ ]:
# Install required packages
%pip install langchain langchain-openai langgraph langchain-community


## 2. Environment Configuration


In [ ]:
import os
from typing import Annotated, Literal
from typing_extensions import TypedDict

# Configure Azure OpenAI environment variables
# Replace these with your actual Azure OpenAI credentials
os.environ["OPENAI_API_VERSION"] = "2023-12-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource-name.openai.azure.com"
os.environ["AZURE_OPENAI_API_KEY"] = "your-azure-openai-api-key"

# Optional: If using Azure Active Directory authentication
# os.environ["OPENAI_API_TYPE"] = "azure_ad"

print("Environment variables configured successfully!")


## 3. Import Required Libraries


In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
from langgraph.graph import StateGraph, END, START
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
import json
from typing import List, Dict, Any


## 4. Define State Schema


In [ ]:
class MessagesState(TypedDict):
    messages: Annotated[List[BaseMessage], "The messages in the conversation"]
    next: str


## 5. Initialize Azure OpenAI Models


In [ ]:
# Initialize Azure OpenAI models for different agents
# Replace deployment names with your actual deployment names

# Main LLM for supervisor and general tasks
llm = AzureChatOpenAI(
    deployment_name="gpt-4",  # Replace with your GPT-4 deployment name
    model_name="gpt-4",
    temperature=0.1,
    max_tokens=1000
)

# Alternative model for specialized tasks (optional)
llm_math = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",  # Replace with your GPT-3.5 deployment name
    model_name="gpt-3.5-turbo",
    temperature=0.0,  # Lower temperature for math tasks
    max_tokens=500
)

print("Azure OpenAI models initialized successfully!")


## 6. Define Tools for Each Agent


In [ ]:
# Research Agent Tools
@tool
def search_web(query: str) -> str:
    """Search the web for information about a given query.
    
    Args:
        query: The search query to look up information about.
    
    Returns:
        A string containing the search results.
    """
    # In a real implementation, you would use a web search API
    # For this example, we'll simulate search results
    return f"Search results for '{query}': This is simulated research data. In a real implementation, you would integrate with a web search API like Serper, Tavily, or Google Search."

@tool
def get_financial_data(company: str, metric: str) -> str:
    """Get financial data for a company.
    
    Args:
        company: The company name or ticker symbol.
        metric: The financial metric to retrieve (e.g., 'revenue', 'gdp', 'market_cap').
    
    Returns:
        A string containing the financial data.
    """
    # Simulated financial data
    if "US" in company.upper() and "GDP" in metric.upper():
        return "US GDP in 2024: $28.7 trillion (simulated data)"
    elif "NEW YORK" in company.upper() and "GDP" in metric.upper():
        return "New York State GDP in 2024: $2.1 trillion (simulated data)"
    else:
        return f"Financial data for {company} - {metric}: Data not available (simulated)"

# Math Agent Tools
@tool
def calculate_percentage(part: float, whole: float) -> str:
    """Calculate the percentage that part represents of the whole.
    
    Args:
        part: The part value.
        whole: The whole value.
    
    Returns:
        A string with the percentage calculation.
    """
    if whole == 0:
        return "Error: Cannot divide by zero"
    percentage = (part / whole) * 100
    return f"{part} is {percentage:.2f}% of {whole}"

@tool
def perform_calculation(expression: str) -> str:
    """Perform a mathematical calculation safely.
    
    Args:
        expression: A mathematical expression to evaluate.
    
    Returns:
        A string with the calculation result.
    """
    try:
        # Simple safe evaluation for basic math
        allowed_chars = set('0123456789+-*/.() ')
        if all(c in allowed_chars for c in expression):
            result = eval(expression)
            return f"{expression} = {result}"
        else:
            return "Error: Expression contains invalid characters"
    except Exception as e:
        return f"Error calculating '{expression}': {str(e)}"

# Writing Agent Tools
@tool
def format_report(data: str, format_type: str = "markdown") -> str:
    """Format data into a structured report.
    
    Args:
        data: The data to format.
        format_type: The format type (markdown, json, plain).
    
    Returns:
        A formatted report string.
    """
    if format_type.lower() == "markdown":
        return f"# Report\n\n{data}\n\n---\n*Generated by Multi-Agent System*"
    elif format_type.lower() == "json":
        return json.dumps({"report": data, "format": format_type}, indent=2)
    else:
        return f"REPORT:\n{data}\n\n---\nGenerated by Multi-Agent System"

print("Tools defined successfully!")
